##### [2022-DataMining]
# Confusion Matrix
####  최민영

In [2]:
import numpy as np
import pandas as pd

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, precision_score, fbeta_score, f1_score

In [3]:
result = pd.read_csv('C:/Users/wl498/Documents/datamine/spotify_knn_15_rbst_seed1.csv')
result['y_pred'] = result['y_prob'].apply(lambda x:x> .5).astype('int16')

In [4]:
result.y_true.eq(result.y_pred).sum()/result.shape[0]

0.7791666666666667

In [5]:
pd.crosstab(result.y_true, result.y_pred)

y_pred    0    1
y_true          
0       611  363
1        61  885

#### Confusion Matrix
**행(row): 실제, 열(column): 예측으로 구성되어 있다.**

* C[1,1]: TP(True Positive)
* C[0,1]: FP(False Positive)
* C[0,0]: TN(True Negative)
* C[1,0]: FN(False Negative)

In [6]:
cm = confusion_matrix(y_true=result.y_true, y_pred = result.y_pred)

In [7]:
tn, fp, fn, tp =  cm[0,0], cm[0,1], cm[1,0], cm[1,1]

In [8]:
print((tp+tn)/cm.sum())
print("accuracy:", accuracy_score(result.y_true, result.y_pred))

0.7791666666666667
accuracy: 0.7791666666666667


In [9]:
result2 = result[['y_true','y_pred']]+1

In [10]:
### Sensitivity(=recall,TPR)
print(tp/cm[1,].sum())
print(recall_score(result.y_true, result.y_pred, pos_label=1))

0.9355179704016914
0.9355179704016914


In [11]:
### Precision(=PPV)
precision_score(result.y_true, result.y_pred, pos_label=1)

0.7091346153846154

In [12]:
### Specificity(=TNR)
print("tnr:", recall_score(result.y_true, result.y_pred, pos_label=0) )

tnr: 0.6273100616016427


In [13]:
### F_beta Score
from statistics import harmonic_mean
recall = recall_score(result.y_true, result.y_pred, pos_label=1)
prc = precision_score(result.y_true, result.y_pred, pos_label=1)
print("f1:", fbeta_score(result.y_true, result.y_pred, beta=1))
print("f1:", f1_score(result.y_true, result.y_pred))
print("f2:", fbeta_score(result.y_true, result.y_pred, beta=2))
print("f0.5:", fbeta_score(result.y_true, result.y_pred, beta=0.5))

f1: 0.8067456700091158
f1: 0.8067456700091158
f2: 0.8793720190779015
f0.5: 0.7452004041764906


In [14]:
### Positive Likelihhod Ratio = tpr/fpr
tpr = recall
tnr = recall_score(result.y_true,result.y_pred, pos_label = 0)
plr = tpr/(1-tnr)
### Negative Likelihood Ratio = fnr/ tnr
fnr = 1-tpr
nlr = fnr/tnr
print(plr,nlr)

2.510177694686632 0.1027913205053234


In [15]:
bacc=(tpr+tnr)/2
bacc

0.781414016001667

### Binary Classification Evaluation Metrics

In [16]:
from sklearn.metrics import classification_report
print(classification_report(result.y_true, result.y_pred))

              precision    recall  f1-score   support

           0       0.91      0.63      0.74       974
           1       0.71      0.94      0.81       946

    accuracy                           0.78      1920
   macro avg       0.81      0.78      0.77      1920
weighted avg       0.81      0.78      0.77      1920



In [17]:
class BCMetrics:
    def __init__(self, y_true, y_pred, pos_label = 1):
        self.y_true = y_true if isinstance(y_true, np.ndarray) else np.array(y_true)
        self.y_pred = y_pred if isinstance(y_pred, np.ndarray) else np.array(y_pred)
        self.labels = np.unique(y_true)
        self.pos_label = pos_label
        self.neg_label = np.delete(self.labels, self.pos_label).item()
        self.metric_func_dicts = {
                            'acc': accuracy_score,
                            'tpr': lambda t,p: recall_score(t, p, pos_label = self.pos_label),
                             'tnr': lambda t,p: recall_score(t, p, pos_label= self.neg_label),
                             'f1': lambda t,p: f1_score(t, p, pos_label= self.pos_label) ,
                             'ppv': lambda t,p: precision_score(t,p, pos_label=self.pos_label),
                             'npv': lambda t,p: precision_score(t,p, pos_label=self.neg_label)
                             }
        
    def cal_metric(self,f, **kwargs):
        return f(self.y_true, self.y_pred, **kwargs)
    
    def evaluate(self, return_df = False):
        result = {f: self.metric_func_dicts[f](self.y_true, self.y_pred) for f in self.metric_func_dicts}
        result['bacc'] = (result['tpr'] + result['tnr'])/2
        result['fpr'] = 1 - result['tpr']
        result['fnr'] = 1 - result['tnr']
        result['plr'] =  result['tpr'] / result['fpr']
        result['nlr'] =  result['tnr'] / result['fnr']
        
        self.result = result
        if return_df:
            return pd.DataFrame({ k: [v] for k, v in result.items()})
        
        return result

In [18]:
bcmetrics = BCMetrics(result.y_true, result.y_pred)
bcmetrics.evaluate(return_df=True)

acc       tpr      tnr        f1       ppv       npv      bacc  \
0  0.779167  0.935518  0.62731  0.806746  0.709135  0.909226  0.781414   

        fpr      fnr        plr       nlr  
0  0.064482  0.37269  14.508197  1.683196